In [1]:
import csv
import os
import pandas as pd
import torch
import torch.optim as optim
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from time import time

In [2]:
def create_csv(root_folder, output_csv):
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['image_filename', 'label'])

        for label in os.listdir(root_folder):
            label_path = os.path.join(root_folder, label)
            if os.path.isdir(label_path):
                for image_filename in os.listdir(label_path):
                    image_path = os.path.join(label, image_filename)
                    label_value = int(label)
                    writer.writerow([image_path, label_value])

# Specify the path to your "Train" and "Test" dataset folders
train_dataset_folder = 'Train/'
test_dataset_folder = 'Test/'

# Create the CSV files
create_csv(train_dataset_folder, "Train/train.csv")
create_csv(test_dataset_folder, "Test/test.csv")

print('CSV files created.')

CSV files created.


In [3]:
class AIDER(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])

        # Use PIL to open and transform the image
        image = Image.open(img_path).convert('RGB')

        label = int(self.img_labels.iloc[idx, 1])  # Convert label to integer
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

        # Convert label to torch.LongTensor
        label = torch.LongTensor([label])
        return image, label

In [4]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=False)
in_features = model.fc.in_features
model.fc = torch.nn.Linear(in_features, 5)

Using cache found in C:\Users\최신우/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\최신우\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\최신우\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\최신우\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\googlenet.py:47: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.

In [5]:
training_data = AIDER(
    annotations_file="Train/train.csv",
    img_dir="Train/",
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
)

test_data = AIDER(
    annotations_file="Test/test.csv",
    img_dir="Test/",
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
)

train_loader = torch.utils.data.DataLoader(training_data, batch_size=32, shuffle=True, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, pin_memory=True)

In [7]:
if torch.cuda.is_available():
    print("GPU is available.")
else:
    print("GPU is not available. Switching to CPU.")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to train the model
def train(model, train_loader, optimizer, criterion, num_epochs=5, print_every=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        initial_time = time()
        prev_time = initial_time
        for i, (inputs, labels) in enumerate(train_loader, 1):
            optimizer.zero_grad()
            
            # Move inputs and labels to GPU
            inputs, labels = inputs.to(device), labels.to(device)

            outputs, *_ = model(inputs)
            loss = criterion(outputs, labels.squeeze())
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Print the loss every print_every batches
            if i % print_every == 0 or i == len(train_loader):
                avg_loss = running_loss / i
                print(f'Epoch {epoch + 1}/{num_epochs}, Batch {i}/{len(train_loader)}, Loss: {avg_loss}, Time taken since last batch: {time() - prev_time:.2f}s, Total time taken: {time() - initial_time:.2f}s')
                prev_time = time()

# Train the model
num_epochs = 20  # You can adjust this value
print_every = 7  # Print loss every 10 batches
train(model, train_loader, optimizer, criterion, num_epochs, print_every)

GPU is not available. Switching to CPU.
Epoch 1/20, Batch 7/161, Loss: 1.2401175924709864, Time taken since last batch: 16.16s, Total time taken: 16.16s
Epoch 1/20, Batch 14/161, Loss: 1.0924789905548096, Time taken since last batch: 14.45s, Total time taken: 30.61s
Epoch 1/20, Batch 21/161, Loss: 1.044010789621444, Time taken since last batch: 14.34s, Total time taken: 44.95s
Epoch 1/20, Batch 28/161, Loss: 1.0078392390693938, Time taken since last batch: 14.59s, Total time taken: 59.54s
Epoch 1/20, Batch 35/161, Loss: 0.9767641970089503, Time taken since last batch: 14.49s, Total time taken: 74.03s
Epoch 1/20, Batch 42/161, Loss: 0.9345507806255704, Time taken since last batch: 14.40s, Total time taken: 88.43s
Epoch 1/20, Batch 49/161, Loss: 0.946445035691164, Time taken since last batch: 14.57s, Total time taken: 103.00s
Epoch 1/20, Batch 56/161, Loss: 0.9332880175539425, Time taken since last batch: 14.55s, Total time taken: 117.55s
Epoch 1/20, Batch 63/161, Loss: 0.924788079564533

In [8]:
# Evaluate the model on the test set
model.eval()
correct = 0
total = 0
predicted_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        # Move inputs and labels to GPU
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        labels = labels.flatten()
        predicted = predicted.flatten()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Append the predicted labels to the list
        predicted_labels.extend(predicted.tolist())

# Convert predicted labels to their corresponding class names
class_mapping = {0: 'normal', 1: 'fire', 2: 'flood', 3: 'collapsed_buildings', 4: 'car_accident'}
predicted_labels = [class_mapping[i] for i in predicted_labels]

# Calculate accuracy
accuracy = correct / total
print(f'Test Accuracy: {100 * accuracy:.2f}%')

Test Accuracy: 84.21%
